In [27]:
# NB25 — repo root & presence
from pathlib import Path

def find_repo_root(start: Path, must_have=("app","notebooks")) -> Path:
    cur = start.resolve()
    for _ in range(7):
        if all((cur/m).exists() for m in must_have): return cur
        cur = cur.parent
    if start.name.lower()=="notebooks" and all((start.parent/m).exists() for m in must_have):
        return start.parent.resolve()
    return start.resolve()

ROOT = find_repo_root(Path.cwd())
print("ROOT:", ROOT)

checks = {
    "app/": (ROOT/"app").exists(),
    "app/api.py": (ROOT/"app"/"api.py").exists(),
    "notebooks/app/api.py": (ROOT/"notebooks"/"app"/"api.py").exists(),
    "artifacts/": (ROOT/"artifacts").exists(),
    "artifacts_crypto/": (ROOT/"artifacts_crypto").exists(),
    "data/df_nb02.csv": (ROOT/"data"/"df_nb02.csv").exists(),
    "data/df_nb02.parquet": (ROOT/"data"/"df_nb02.parquet").exists(),
}
for k,v in checks.items():
    print(f"{k:24s}:", v)


ROOT: C:\.projects\stock-direction-ml\stock-direction-ml-2
app/                    : True
app/api.py              : True
notebooks/app/api.py    : False
artifacts/              : True
artifacts_crypto/       : True
data/df_nb02.csv        : True
data/df_nb02.parquet    : False


In [28]:
# NB25 — write/update selective_config.json (global τ + inversion)
import json
from pathlib import Path

cfg_path = ROOT/"artifacts"/"selective_config.json"
cfg_path.parent.mkdir(parents=True, exist_ok=True)

# You can tweak these later if needed:
cfg = {
    "asset_class": "equity",
    "target_precision": 0.90,
    "global_tau": 0.85,         # chosen from your NB25 sweep
    "invert_proba": True,       # use 1 - p as the decision score
    "notes": "Selective mode: high precision, lower coverage."
}

# Merge if file exists, else write fresh
if cfg_path.exists():
    old = json.loads(cfg_path.read_text(encoding="utf-8"))
    old.update(cfg)
    cfg = old

cfg_path.write_text(json.dumps(cfg, indent=2), encoding="utf-8")
print("Wrote:", cfg_path.resolve())
print(cfg)


Wrote: C:\.projects\stock-direction-ml\stock-direction-ml-2\artifacts\selective_config.json
{'asset_class': 'equity', 'target_precision': 0.9, 'global_tau': 0.85, 'invert_proba': True, 'notes': 'Selective mode: high precision, lower coverage.'}


In [29]:
# NB25 — append selective wrapper to api.py (idempotent)
from pathlib import Path
import sys, types, importlib.util

candidates = [ROOT/"app"/"api.py", ROOT/"notebooks"/"app"/"api.py"]
api_path = next((p for p in candidates if p.exists()), None)
print("api.py:", api_path if api_path else "NOT FOUND")
if api_path is None:
    raise FileNotFoundError("No api.py found in app/ or notebooks/app/")

src = api_path.read_text(encoding="utf-8")
marker = "# --- selective override wrapper (auto-added by NB25) ---"

if marker not in src:
    wrapper = "\n\n" + marker + "\n" + '''
try:
    _sel_wrapper_applied  # guard
except NameError:
    _orig_predict_core = predict_core

    def predict_core(asset_class="equity", source="repo", ticker=None,
                     start=None, end=None, fee_bps=5, tau=None):
        pred, meta = _orig_predict_core(asset_class=asset_class, source=source, ticker=ticker,
                                        start=start, end=end, fee_bps=fee_bps, tau=tau)

        import json, numpy as _np, pandas as _pd
        from pathlib import Path as _P

        # Locate repo root for artifacts
        try:
            base = ROOT  # provided by app.config
        except NameError:
            base = _P(__file__).resolve().parent.parent  # app/ → repo root

        # Load selective settings
        sel = {}
        cfgp = base / "artifacts" / "selective_config.json"
        if cfgp.exists():
            try: sel = json.loads(cfgp.read_text(encoding="utf-8"))
            except Exception: sel = {}

        invert = bool(sel.get("invert_proba", False))
        tau_default = float(sel.get("global_tau", meta.get("tau_used", 0.59)))
        tau_used = float(tau) if tau is not None else tau_default

        # Normalize to DataFrame for post-processing
        if isinstance(pred, list):
            dfp = _pd.DataFrame(pred); ret_type = "list"
        else:
            dfp = pred.copy(); ret_type = "df"

        # If proba present → apply inversion + re-threshold, and sync tau_used column
        if "proba" in dfp.columns:
            proba = _pd.to_numeric(dfp["proba"], errors="coerce").to_numpy()
            proba = _np.clip(proba, 1e-6, 1-1e-6)
            if invert:
                proba = 1.0 - proba
                dfp["proba"] = proba
            dfp["signal"] = (proba >= tau_used).astype(int)
        dfp["tau_used"] = float(tau_used)
        meta["tau_used"] = tau_used

        return (dfp.to_dict(orient="records"), meta) if ret_type=="list" else (dfp, meta)

    _sel_wrapper_applied = True
'''
    api_path.write_text(src + wrapper, encoding="utf-8")
    print("Wrapper appended →", api_path)
else:
    print("Wrapper already present; no change.")

# Load module from file path (works regardless of sys.path)
spec = importlib.util.spec_from_file_location("app.api", str(api_path))
pkg = types.ModuleType("app"); pkg.__path__ = [str(api_path.parent)]
sys.modules["app"] = pkg
mod = importlib.util.module_from_spec(spec)
sys.modules["app.api"] = mod
spec.loader.exec_module(mod)
api = mod
print("Loaded:", api)


api.py: C:\.projects\stock-direction-ml\stock-direction-ml-2\app\api.py
Wrapper already present; no change.
Loaded: <module 'app.api' from 'C:\\.projects\\stock-direction-ml\\stock-direction-ml-2\\app\\api.py'>


In [30]:
# NB25 — smoke test (equity, repo source)
import pandas as pd

pred, meta = api.predict_core(
    asset_class="equity",
    source="repo",
    ticker=None,   # auto-picks AAPL if present else majority ticker
    start=None, end=None,
    fee_bps=5, tau=None
)

print("tau_used(meta):", meta.get("tau_used"), "| return type:", type(pred))
df_eq = pd.DataFrame(pred) if isinstance(pred, list) else pred
print(df_eq.tail(3)[["date","ticker","proba","signal","tau_used"]])
print("Unique tau_used in rows:", df_eq["tau_used"].unique()[:5])


tau_used(meta): 0.85 | return type: <class 'list'>
            date ticker     proba  signal  tau_used
2683  2025-10-08   AAPL  0.305639       0      0.85
2684  2025-10-09   AAPL  0.301950       0      0.85
2685  2025-10-10   AAPL  0.359084       0      0.85
Unique tau_used in rows: [0.85]


In [31]:
# NB25 — smoke test (crypto, fetch source) + run instructions
import pandas as pd

try:
    pred, meta = api.predict_core(
        asset_class="crypto",
        source="fetch",
        ticker="BTC-USD",
        start=str(pd.Timestamp.today().date().replace(year=pd.Timestamp.today().year-1)),
        end=None,
        fee_bps=5,
        tau=None
    )
    df_cr = pd.DataFrame(pred) if isinstance(pred, list) else pred
    print("CRYPTO tau_used(meta):", meta.get("tau_used"), "| rows:", len(df_cr))
    print(df_cr.tail(3)[["date","ticker","proba","signal","tau_used"]])
except Exception as e:
    print("Crypto fetch skipped:", e)

print("\nRun Streamlit locally from REPO ROOT:")
print("  streamlit run app/streamlit_app.py")
print("\nRun API locally (optional):")
print("  uvicorn app.api:app --reload --port=8000")


CRYPTO tau_used(meta): 0.85 | rows: 256
           date   ticker     proba  signal  tau_used
253  2025-10-27  BTC-USD  0.508027       0      0.85
254  2025-10-28  BTC-USD  0.504568       0      0.85
255  2025-10-29  BTC-USD  0.492516       0      0.85

Run Streamlit locally from REPO ROOT:
  streamlit run app/streamlit_app.py

Run API locally (optional):
  uvicorn app.api:app --reload --port=8000
